# Pytree 사용하기

JAX는 딕셔너리, 배열, 중첩 리스트, 딕셔너리 리스트 혼합 중첩 등 다양한 구조에 대한 연산을 지원하는데, 이러한 구조들을 JAX에서는 pytree라고 부릅니다. 해당 문서에서는 pytree를 잘 사용하는 법과 실수할 수 있는 부분들을 짚고 넘어가겠습니다.

## pytree란

[공식 도큐먼트](https://jax.readthedocs.io/en/latest/working-with-pytrees.html) 에서 pytree에 대해 설명한 글을 직역하면 다음과 같습니다(어떻게 더 잘 번역을 해야할지 감이 안와 공식 도큐먼트를 그대로 인용합니다.):

*파이 트리는 컨테이너 형태의 Python 객체들로 구성된 구조입니다. “리프(leaf)” 파이 트리와 더 많은 파이 트리들로 이루어질 수 있습니다. 파이 트리는 리스트, 튜플, 사전을 포함할 수 있습니다. 리프(leaf)는 배열과 같이 파이 트리가 아닌 모든 것을 의미하지만, 단일 리프도 파이 트리입니다.*

머신러닝의 관점에서 pytree는 다음과 같은 정보를 포함할 수 있습니다:

- 모델 파라메터
- 데이터셋
- 강화학습 문제에서의 에이전트 observation

아래는 간단한 pytree 예제입니다. example_trees 라는 pytree object를 만들어 보겠습니다:

In [1]:
import jax
import jax.numpy as jnp

example_trees = [
    [1, 'a', object()],
    (1, (2, 3), ()),
    [1, {'k1': 2, 'k2': (3, 4)}, 5],
    {'a': 2, 'b': (2, 3)},
    jnp.array([1, 2, 3]),
]

`jax.tree.leaves()`는 리프 노드들을 flatten한 값을 추출합니다. 아래는 `jax.tree.leaves()` 없이 출력한 결과입니다:

In [2]:
for pytree in example_trees:
    print(f"{repr(pytree):<45} has {len(pytree)} leaves: {pytree} leavers: {pytree}")

[1, 'a', <object object at 0x105933620>]      has 3 leaves: [1, 'a', <object object at 0x105933620>] leavers: [1, 'a', <object object at 0x105933620>]
(1, (2, 3), ())                               has 3 leaves: (1, (2, 3), ()) leavers: (1, (2, 3), ())
[1, {'k1': 2, 'k2': (3, 4)}, 5]               has 3 leaves: [1, {'k1': 2, 'k2': (3, 4)}, 5] leavers: [1, {'k1': 2, 'k2': (3, 4)}, 5]
{'a': 2, 'b': (2, 3)}                         has 2 leaves: {'a': 2, 'b': (2, 3)} leavers: {'a': 2, 'b': (2, 3)}
Array([1, 2, 3], dtype=int32)                 has 3 leaves: [1 2 3] leavers: [1 2 3]


아래 코드는 `jax.tree.leaves()`를 사용하여 리프 노드를 출력한 결과입니다:

In [3]:
for pytree in example_trees:
    leaves = jax.tree.leaves(pytree)
    print(f"{repr(pytree):<45} has {len(leaves)} leaves: {leaves}")

[1, 'a', <object object at 0x105933620>]      has 3 leaves: [1, 'a', <object object at 0x105933620>]
(1, (2, 3), ())                               has 3 leaves: [1, 2, 3]
[1, {'k1': 2, 'k2': (3, 4)}, 5]               has 5 leaves: [1, 2, 3, 4, 5]
{'a': 2, 'b': (2, 3)}                         has 3 leaves: [2, 2, 3]
Array([1, 2, 3], dtype=int32)                 has 1 leaves: [Array([1, 2, 3], dtype=int32)]


파이썬 컨테이너 형태의 트리 구조로 이루어진 객체들은 모두 pytree로 취급할 수 있습니다. JAX는 pytree 레지스트리를 가지고 있고 레지스트리에 등록된 객체를 pytree로 인식하는데, 리스트, 딕셔너리, 튜플은 기본적으로 pytree 레지스트리에 등록되어 있습니다. 클래스 형태 또한 pytree 레지스트리에 등록한다면 pytree로 인식되게 할 수 있으며, 레지스트리에 등록되지 않은 모든 객체들은 리프 노드로 취급됩니다.

아래 [사용자 정의 pytree](#사용자-정의-pytree) 섹션에서 커스텀 pytree를 만들고 pytree 레지스트리에 등록하는 방법을 알아보겠습니다.

## 대표적인 pytree 함수들

JAX에는 pytree를 다루기 위한 다양한 함수들이 있으며, `jax.tree_util` 아래에 위치하고 있습니다. 하지만 편의를 위해 `jax.tree`로도 접근할 수 있습니다(Aliasing).

### `jax.tree.map`

가장 흔하게 사용되는 pytree 함수는 `jax.tree.map()`입니다. 파이썬 기본함수 `map`과 유사하게 작동하지만 pytree 내부 모든 객체에 대해서 적용됩니다:

In [4]:
list_of_lists = [
    [1, 2, 3],
    [1, 2],
    [1, 2, 3, 4],
]

jax.tree.map(lambda x: x*2, list_of_lists)

[[2, 4, 6], [2, 4], [2, 4, 6, 8]]

In [5]:
another_list_of_lists = list_of_lists
jax.tree.map(lambda x, y: x+y, list_of_lists, another_list_of_lists)

[[2, 4, 6], [2, 4], [2, 4, 6, 8]]

여러개의 인자를 사용할 떄 연산이 이루어지는 pytree간의 형태는 같아야 합니다. 

### 예시 1: `jax.tree.map`로 딥러닝 모델 파라메터 다루기

아래 코드와 같이 간단한 신경망(MLP)을 다룰 때, 신경망의 파라메를 pytree로 정의할 수 있습니다:

In [6]:
import numpy as np

def init_mlp_params(layer_widths):
  params = []
  for n_in, n_out in zip(layer_widths[:-1], layer_widths[1:]):
    params.append(
        dict(weights=np.random.normal(size=(n_in, n_out)) * np.sqrt(2/n_in),
             biases=np.ones(shape=(n_out,))
            )
    )
  return params

params = init_mlp_params([1, 128, 128, 1])

> **NOTE**
> 위의 공식 예제 코드에서 신경망 파라메터를 NumPy를 써서 정의한 것을 볼 수 있는데, 왜 이렇게 한지는 모르겠습니다. numpy 배열을 넘겨줘도 JIT 컴파일은 잘 되는데, 제 생각에는 NumPy 특성상 JAX보다 더 직관적인 코드를 보여줄 수 있어서 가독성 측면에서 Jax.Array 대신 Numpy 배열을 사용한 것 같습니다.

In [7]:
jax.tree.map(lambda x: x.shape, params)

[{'biases': (128,), 'weights': (1, 128)},
 {'biases': (128,), 'weights': (128, 128)},
 {'biases': (1,), 'weights': (128, 1)}]

In [8]:
# Define the forward pass.
def forward(params, x):
  *hidden, last = params
  for layer in hidden:
    x = jax.nn.relu(x @ layer['weights'] + layer['biases'])
  return x @ last['weights'] + last['biases']

# Define the loss function.
def loss_fn(params, x, y):
  return jnp.mean((forward(params, x) - y) ** 2)

# Set the learning rate.
LEARNING_RATE = 0.0001

# Using the stochastic gradient descent, define the parameter update function.
# Apply `@jax.jit` for JIT compilation (speed).
@jax.jit
def update(params, x, y):
  # Calculate the gradients with `jax.grad`.
  grads = jax.grad(loss_fn)(params, x, y)
  # Note that `grads` is a pytree with the same structure as `params`.
  # `jax.grad` is one of many JAX functions that has
  # built-in support for pytrees.
  # This is useful - you can apply the SGD update using JAX pytree utilities.
  return jax.tree.map(
      lambda p, g: p - LEARNING_RATE * g, params, grads
  ), grads

## 사용자 정의 pytree

`jax.tree_util.register_pytree_node()`를 사용하여 원하는 pytree 노드를 만들 수 있습니다.

만약 별다른 처리 없이 사용자 정의 클래스를 pytree 안에 포함시킨다면, 비록 클래스 안에 pytree 객체가 포함되어 있다고 하더라도 JAX가 인식하지 못합니다. 아래와 같이 사용자 정의 클래스를 정의해 봅시다:

In [9]:
class Special(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
jax.tree.leaves([
    Special(0, 1),
    Special(2, 4),
])

[<__main__.Special at 0x105ad2ad0>, <__main__.Special at 0x117d78e50>]

때문에 아래와 같은 연산은 오류가 나게 됩니다. 클래스 내부 변수를 리프처럼 다루려고 하는데, JAX가 보기에는 클래스 자체가 리프가 되기 때문입니다:

In [10]:
jax.tree.map(lambda x: x + 1,
    [
        Special(0, 1),
        Special(2, 4),
    ])

TypeError: unsupported operand type(s) for +: 'Special' and 'int'

사용자 정의 클래스를 pytree 내부에서 노드로 작용하게끔 만들고 싶다면 JAX의 pytree 레지스트리에 등록을 하면 됩니다. 그럼 해당 클래스는 전역 레지스트리에 등록되게 됩니다. 레지스트리에 등록된 타입은 재귀적으로 탐색하여 찾게 됩니다.

아래와 같이 `jax.tree_util.register_pytree_node()`를 사용하여 레지스트리에 등록할 수 있습니다:

In [11]:
from jax.tree_util import register_pytree_node

class RegisteredSpecial(Special):
    def __repr__(self):
        return "RegisteredSpecial(x={}, y={})".format(self.x, self.y)
    
def special_flatten(v):
    """사용자 정의 클래스의 Flatten 연산을 정의해 줍니다.

    Args:
        v: flatten이 일어날 사용자 정의 클래스
    
    Returns:
        flatten이 적용된 iterable 객채와, 이후 unflatten이 일어날 때 필요한 정보를 담은 auxiliary를 리턴합니다.
        auxiliary는 이후 unflatten을 위해 treedef에 저장됩니다. 예를 들어 딕셔너리의 키값이 auxiliary가 될 수 있습니다. 
    """
    children = (v.x, v.y)
    aux_data = None
    return (children, aux_data)

def special_unflatten(aux_data, children):
    """사용자 정의 클래스의 Unflatten 연산을 정의해 줍니다.

    Args:
        aux_data: 시용자 클래스 객체가 flatten되면서 정의된 데이터
        children: flatten된 사용자 정의 객체
    
    Returns:
        children과 aux_data를 통해 unflatten(재구성)된 사용자 정의 객체
    """
    return RegisteredSpecial(*children)

# 전역 레지스트리에 등록
register_pytree_node(
    RegisteredSpecial,
    special_flatten,    # Jax에 children node가 뭔지 알려줌
    special_unflatten,  # Jax에 어떻게 RegisteredSpecial객체를 unflatten 시키는지 알려줌
)

이제 사용자 정의 클래스를 다음과 같이 노드로써 사용 가능합니다:

In [12]:
jax.tree.map(lambda x: x + 1,
    [
        RegisteredSpecial(0, 1),
        RegisteredSpecial(2, 4),
    ])

[RegisteredSpecial(x=1, y=2), RegisteredSpecial(x=3, y=5)]

모던 파이썬에는 사용자 정의 객체를 더욱 편리하게 만들 수 있도록 하는 편의기능이 많이 있습니다. 어떤건 JAX와 잘 작동하면서도 어떤건 주의가 필요합니다.

예를 들어 파이썬의 `NamedTuple` 서브클래스는 따로 레지스트리에 등록할 필요가 없이 pytree의 node로써 사용할 수 있습니다:

In [13]:
from typing import NamedTuple, Any

class MyOtherContainer(NamedTuple):
    name: str
    a: Any
    b: Any
    c: Any

# NamedTuple은 pytree의 Node로 인식되기 때문에
# 별다른 레지스트리 등록 없이 사용 가능
jax.tree.leaves([
    MyOtherContainer('Alice', 1, 2, 3),
    MyOtherContainer('Bob', 4, 5, 6),
])

['Alice', 1, 2, 3, 'Bob', 4, 5, 6]

## 변환함수와 pytree

JAX의 모든 변환함수는 pytree를 통한 입출력이 가능하게끔 설계되어 있습니다.

`jax.vmap()`에서는 `in_axis`와 `out_axis`를 사용하여 인풋값과 아웃풋값에 대한 컨트롤을 할 수 있는데, 다른 몇몇 변환함수도 이러한 옵션 인자를 받는 경우가 있습니다. 이러한 파라메터는 모두 pytree에도 적용됩니다.
옵션 인자로 들어가는 값은 인풋으로 들어가는 pytree와 같은 구조를 가지고 있어야 합니다.  

예를 들어, 다음 입력을 `jax.vmap()`에 전달한다고 가정합니다 (함수의 입력 인수는 튜플로 간주됨):

In [ ]:
vmap(f, in_axes=(a1, {"k1": a2, "k2": a3}))

그런 다음 다음과 같은 `in_axes` 파이트리를 사용하여 `k2` 인수만 매핑되도록 지정할 수 있습니다. 아래에서 0으로 지정된 `k2`에 대해서는 0번째 차원을 기준으로 매핑되며, 나머지 값들에 대해서는 vmap이 적용되지 않습니다.

In [ ]:
vmap(f, in_axes=(None, {"k1": None, "k2": 0}))

위와 같이 옵션 인자로 들어가는 값은 인풋값의 형태와 일치해야 합니다. 

하지만 아래와 코드와 같은 편의 기능도 제공하는데, 아래와 같이 작성하면 딕셔너리 전체의 값을 지정할 수도 있습니다

In [ ]:
vmap(f, in_axes=(None, 0))  # (None, {"k1": 0, "k2": 0})와 동일

만약 모든 값에 대해서 동일하게 mapping해야 한다면 스칼라값만 넣어도 전체 값에 대해서 적용됩니다:

In [ ]:
vmap(f, in_axes=0)  # (0, {"k1": 0, "k2": 0})와 동일

## `key path`

pytree의 리프들은 *key path*를 같습니다. 리프의 key path는 key로 이루어진 리스트인데, 이때 리스트의 길이는 pytree에서 위치한 리프의 깊이와 같습니다. 각 키는 리프가 위치한 노드의 타입과, 노트 내부의 인덱스를 나타네는 hashable 객체입니다. 즉, 딕셔너리 타입의 key와 튜플 타입의 key는 다른 타입입니다.

아래 함수들은 `jax.tree.util.*` 에 있는, key path를 다루는 유틸리티 함수들입니다:

- `jax.tree_util.tree_flatten_with_path()`: `jax.tree.flatten()`와 같은 역할이지만 key path를 리턴합니다.
- `jax.tree_util.tree_map_with_path()`: `jax.tree.map()`과 같은 역할이지만 key path 또한 인자로 받습니다.
- `jax.tree_util.keystr()` key path를 인자로 받고 더욱 읽기 쉬운 형태로 변환한 값을 반환합니다.

아래 예시는 디버깅을 위해 리프 값들의 정보를 프린트해 보는 상황입니다:

In [14]:
import collections

ATuple = collections.namedtuple("ATuple", ('name'))

tree = [1, {'k1': 2, 'k2': (3, 4)}, ATuple('foo')]
flattened, _ = jax.tree_util.tree_flatten_with_path(tree)

for key_path, value in flattened:
  print(f'Value of tree{jax.tree_util.keystr(key_path)}: {value}')

Value of tree[0]: 1
Value of tree[1]['k1']: 2
Value of tree[1]['k2'][0]: 3
Value of tree[1]['k2'][1]: 4
Value of tree[2].name: foo


JAX는 기본적인 노트 타입들에 대해서 default key type을 가지고 있습니다:

- `SequenceKey(idx: int)`: 리스트와 튜플을 위한 타입
- `DictKey(key: Hashable)`: 딕셔너리를 위한 타입
- `GetAttrKey(name: str)`: `namedtuple` 및 사용자 정의 타입을 위한 타입(다음 섹션에서 자세히 다룸)

사용자 정의 노드를 위한 key type을 직접 만들 수 있습니다. 만약  `__str__()` 함수도 오버라이딩 되어 있다면 `jax.tree_util.keystr()`이 자연스럽게 작동할 것입니다:

In [15]:
for key_path, _ in flattened:
  print(f'Key path of tree{jax.tree_util.keystr(key_path)}: {repr(key_path)}')

Key path of tree[0]: (SequenceKey(idx=0),)
Key path of tree[1]['k1']: (SequenceKey(idx=1), DictKey(key='k1'))
Key path of tree[1]['k2'][0]: (SequenceKey(idx=1), DictKey(key='k2'), SequenceKey(idx=0))
Key path of tree[1]['k2'][1]: (SequenceKey(idx=1), DictKey(key='k2'), SequenceKey(idx=1))
Key path of tree[2].name: (SequenceKey(idx=2), GetAttrKey(name='name'))


## 흔한 실수

해당 절에서는 JAX pytree를 사용할 때 할 수 있는 실수들에 대해서 살펴보겠습니다.

### 노드를 리프로 착각하여 사용하는 경우

In [16]:
a_tree = [jnp.zeros((2, 3)), jnp.zeros((3, 4))]

# 0이 아니라 1로 채워진 pytree를 만드려고 시도하는 코드
shapes = jax.tree.map(lambda x: x.shape, a_tree)
jax.tree.map(jnp.ones, shapes)

[(Array([1., 1.], dtype=float32), Array([1., 1., 1.], dtype=float32)),
 (Array([1., 1., 1.], dtype=float32), Array([1., 1., 1., 1.], dtype=float32))]

사용자는 `[jnp.ones((2, 3)), jnp.ones((3, 4))]` 와 같은 pytree가 리턴되기를 기대했을 겁니다. 하지만 최종 결과물은 `[jnp.ones(2), jnp.ones(3), jnp.ones(3), jnp.ones(4)]`가 되었습니다.

위 코드에서 배열의 `shape`은 튜플이며, 이는 pytree에서 노드로 취급됩니다. 즉, 저희는 튜플값 (2, 3)이 리프로 작동하길 기대했지만 실재 JAX는 (2, 3)은 노드이며, 튜플 안의 값 2, 3을 각각 리프로 이해한 것입니다. 때문에 마지막 코드에서 `jax.tree.map()` 연산이 일어날 떄 `(2, 3)`에 대해서 `jnp.ones` 함수를 실행시킨 것이 아니라 `2`, `3` 각각에 대해서 `jax.tree.map()` 연산이 일어나게 된 것입니다.

위와 같은 문제를 해결하는 방법이야 다양하겠지만 2가지 보편적으로 사용되는 방법은 다음과 같습니다:

- 연산의 중간 과정에서 `jax.tree.map()`연산이 일어나지 않도록 코드 수정
- 튜플 부분을 NumPy 배열(`np.array`) 혹은 JAX NunPy 배열 (`jnp.array`)로 수정하여 JAX가 리프로 이해하도록 수정

### `jax.tree_util` 함수에서 `None` 값을 다룰 경우

`jax.tree_util` 함수들은 `None` 값을 하나의 리프로 보는 것이 아니라 **노드의 부재**로 인식합니다:

In [17]:
jax.tree.leaves([None, None, None])

[]

`None` 값을 리프로 다루고 싶다면, `is_leaf` 인자를 사용해야 합니다:

In [18]:
jax.tree.leaves([None, None, None], is_leaf=lambda x: x is None)

[None, None, None]